In [153]:
import os, sys
import glob
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser

#Covarage matrix as csv file without qoutes, seperated by commas
file = "/home/pato/Skrivebord/HPV_subtyping/FASTQ/covMatrix.csv"
amplPos = "/home/pato/Skrivebord/HPV_subtyping/FASTQ/amplPos.bed"
fastqFiles = "/home/pato/Skrivebord/HPV_subtyping/FASTQ"

# Housegenes to check and remove
houseGeneEndRow = 5
typeTier=1

# Location of all valid reference names that can be given
path = "/home/pato/Skrivebord/HPV_subtyping/References"

# Location of where to save output files
refSavePath = "/home/pato/Skrivebord/HPV_subtyping/Scripts/Python/autoDetTest"
logFile = "/home/pato/Skrivebord/HPV_subtyping/Results/testlog.txt"


faF = [f for f in listdir(path) if isfile(join(path, f))]
faFiles = []
for i in faF:
    if "fasta.fai" not in i:
        faFiles.append(i)
df = pd.read_csv(file,sep=",")

housegenes = list(df.iloc[0:int(houseGeneEndRow),0])


In [180]:
excludeCols = ['Sort column', 'Gene', 'contig_srt', 'contig_end', 'Target']
sampleList = []
discardedList = []

for col in df.columns:
  
    if col not in excludeCols:

        # Checking housegenes, first get total cov of them
        hgTot = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            hgTot+= int(hg[col])

        # Now finding outliers in housegene coverage
        notEnoughCov = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            geneCov = int(hg[col])

            # set min cov on housegene to not count as failed
            if geneCov < hgTot*0.1:
                notEnoughCov+=1
            
        # discard sample if too many low housegenes
        if notEnoughCov < 4:
            sampleList.append(col)
        else:
            discardedList.append(col)
discardedList

[]

In [181]:
# Now remove housegenes from samples
ddf = df[~df['Gene'].isin(housegenes)]

In [182]:
ampl_df = pd.read_csv(amplPos,sep="\t", header = None)

In [183]:
# Getting amplicon id
amplList = []
for i in ddf['Gene']:
    amplList.append(i.split("_")[-1])

ddf['Target'] = amplList

<ipython-input-183-a15c38559d55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['Target'] = amplList


In [184]:
ddf[['Type', 'Gene']] = ddf['Gene'].str.split('_', 1, expand=True)
# Removing Ampl from gene
# Getting amplicon id
GeneList = []
for i in ddf['Gene']:
    GeneList.append(i.split("AMPL")[~-1])

ddf['Gene'] = GeneList


/home/pato/.local/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-184-5b023aa00500>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['Gene'] = GeneList


In [160]:
ampl = pd.DataFrame()

In [161]:
ampl['Target'] = ampl_df[3]
ampl['contig_srt'] = ampl_df[1]
ampl['contig_end'] = ampl_df[2]

In [162]:
ddf = ddf.merge(ampl,on='Target')

In [168]:
ddf[93:101]

,Gene,IonXpress_001,IonXpress_002,IonXpress_003,IonXpress_004,IonXpress_005,IonXpress_006,IonXpress_007,IonXpress_008,IonXpress_009,...,IonXpress_064,IonXpress_065,IonXpress_066,IonXpress_067,IonXpress_068,IonXpress_069,Target,Type,contig_srt,contig_end
93,E2_2_,0,0,0,0,0,0,0,62,0,...,0,0,0,0,0,0,AMPL4425940,HPV45,3334,3414
94,E2_3_,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,AMPL4425906,HPV45,3451,3546
95,L1_,0,0,0,0,0,0,0,81,0,...,0,0,0,0,0,0,AMPL4426041,HPV45,6534,6617
96,E6_1_,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,AMPL4426076,HPV51,182,264
97,E6_2_,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,AMPL4425895,HPV51,349,435
98,E7_1_,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,AMPL4426086,HPV51,602,678
99,E7_2_,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,AMPL4425957,HPV51,734,818
100,E2_1_,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,AMPL4426083,HPV51,2864,2951


In [163]:
# Look for deletions (possible integrations of the genome) by finding regions of very low coverage
hpvTypes = pd.unique(ddf['Type'])

for type in hpvTypes:
    filtered = ddf[ddf['Type'] == type]

In [169]:
def savePos(hpvtype,startpos,stoppos):
    # save found startpos and endpos
            if 'startpos' in globals():
                if 'stoppos' not in globals():
                    stoppos = lastrow['contig_end']

                # create key if it does not exist:
                if sample in delJson.keys():
                    pass
                else:    
                    delJson[sample] = {
                    'del' : []
                    }

                delJson[sample]['del'].append([hpvtype,startpos,stoppos])



In [170]:
delJson = {}

for sample in sampleList:
    for hpvtype in hpvTypes:
        filtered = ddf[ddf['Type'] == hpvtype]
        leng = len(filtered[sample])
        sum = filtered[sample].sum()
        
        # If sum of HPV type cover is less than n, do not call an integration
        if sum < 50:
            continue

        avg = sum / leng
        if avg > 50:
            cutoff = 50
        else:
            cutoff = avg*0.05

        # for sample in sampleList:#
        filtered = filtered.reset_index()
        endRow=len(filtered)-1
        lastIdx = 0
        for index, row in filtered.iterrows():
            if row[sample] < cutoff:
                # Check that this is not the first amplicon with reads. 
                # I.e reads have been 0 0 0 25 63, and then a deletion is called
                # infront of the 25

                # Check if last index contained low cov (potential deletion)
                if (lastIdx == index - 1) or (lastIdx == 0):

                    if 'startpos' not in locals():
                        startpos = row['contig_srt']
                    stoppos = row['contig_end']
                    lastIdx = index
                    
                    # If on last position and it passed check for low cov, save
                    if index == endRow:
                        savePos(hpvtype,startpos,stoppos)
                        del startpos, stoppos


                else:
                    # This is start of a new deletion
                    startpos = row['contig_srt']
                    lastrow = row
                    lastIdx = index

                    if "stoppos" in locals():
                        del stoppos 

            else:
                # save found startpos and endpos
                if 'startpos' in locals():
                    if 'stoppos' not in locals():
                        stoppos = lastrow['contig_end']

                    savePos(hpvtype,startpos,stoppos)
                    del startpos, stoppos

In [171]:
delJson

{'IonXpress_003': {'del': [['HPV45', 194, 459], ['HPV45', 2878, 6617]]},
 'IonXpress_006': {'del': [['HPV58', 744, 830]]},
 'IonXpress_008': {'del': [['HPV45', 194, 459],
   ['HPV45', 2878, 2958],
   ['HPV45', 3451, 3546]]},
 'IonXpress_012': {'del': [['HPV16', 149, 234],
   ['HPV16', 729, 808],
   ['HPV16', 3345, 3438]]},
 'IonXpress_015': {'del': [['HPV33', 383, 663], ['HPV33', 2853, 3558]]},
 'IonXpress_018': {'del': [['HPV11', 2894, 6780]]},
 'IonXpress_020': {'del': [['HPV39', 194, 287]]},
 'IonXpress_024': {'del': [['HPV45', 2878, 2958]]},
 'IonXpress_029': {'del': [['HPV16', 149, 234],
   ['HPV16', 575, 665],
   ['HPV16', 2870, 3438]]},
 'IonXpress_030': {'del': [['HPV33', 2853, 6598]]},
 'IonXpress_031': {'del': [['HPV35', 169, 257], ['HPV35', 3323, 3405]]},
 'IonXpress_037': {'del': [['HPV45', 2878, 6617]]},
 'IonXpress_043': {'del': [['HPV33', 383, 663], ['HPV33', 3339, 3423]]},
 'IonXpress_046': {'del': [['HPV18', 130, 220], ['HPV18', 2857, 3676]]},
 'IonXpress_047': {'del':

In [177]:
integrationDF = pd.DataFrame(delJson)

In [178]:
integrationDF.transpose()

,del
IonXpress_003,"[[HPV45, 194, 459], [HPV45, 2878, 6617]]"
IonXpress_006,"[[HPV58, 744, 830]]"
IonXpress_008,"[[HPV45, 194, 459], [HPV45, 2878, 2958], [HPV4..."
IonXpress_012,"[[HPV16, 149, 234], [HPV16, 729, 808], [HPV16,..."
IonXpress_015,"[[HPV33, 383, 663], [HPV33, 2853, 3558]]"
IonXpress_018,"[[HPV11, 2894, 6780]]"
IonXpress_020,"[[HPV39, 194, 287]]"
IonXpress_024,"[[HPV45, 2878, 2958]]"
IonXpress_029,"[[HPV16, 149, 234], [HPV16, 575, 665], [HPV16,..."
IonXpress_030,"[[HPV33, 2853, 6598]]"


In [142]:
filename = refSavePath+"/PossibleIntegrations.txt"

In [143]:
integrationDF.to_csv(filename)